# Crop scenes output from ASF hyp3 for Mintpy

In [1]:
import os
import matplotlib
import numpy as np
import shapely
import geopandas as gpd
import rasterio as rio
import rasterio.mask 
import shutil

In [3]:
# load aoi
aoi = 'C:/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/shapefiles/swiss_testv0.shp'
aoi_gdf = gpd.read_file(aoi)
aoi_gdf_proj = aoi_gdf.to_crs('EPSG:32632')

C:\Users\qbren\miniconda3\envs\gda\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [5]:
# load, crop, and save rasters

dirs = os.listdir('/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_raw')


for idir in dirs:
    cwd = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_raw/{idir}'
    os.chdir(cwd)
    allfiles = os.listdir(cwd)
    tif_list = []
    for fn in allfiles:
        if fn[-4:] == '.tif':
            tif_list.append(fn)
    os.mkdir(f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_crop/{idir}')
    for tif in tif_list:
        tif_fn = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_raw/{idir}/{tif}'
        src = rio.open(tif_fn)
        print("Writing: ", tif)
        out_image, out_transform = rasterio.mask.mask(src, aoi_gdf_proj.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
        out_fn = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_crop/{idir}/{tif}'
        with rio.open(out_fn, "w", **out_meta) as dest:
            dest.write(out_image)
        src.close()
        
    for fn in allfiles:
        if fn[-4:] == '.txt' and fn[-7:] != '.md.txt':
            text_fn = fn
            shutil.copy(f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_raw/{idir}/{text_fn}', f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_crop/{idir}/{text_fn}')    

Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_amp.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_corr.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_dem.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_los_disp.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_lv_phi.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_lv_theta.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_unw_phase.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_vert_disp.tif
Writing:  S1AA_20150702T172305_20150714T172305_VVP012_INT40_G_ueF_A743_water_mask.tif
Writing:  S1AA_20150702T172305_20150726T172306_VVP024_INT40_G_ueF_A8ED_amp.tif
Writing:  S1AA_20150702T172305_20150726T172306_VVP024_INT40_G_ueF_A8ED_corr.tif
Writing:  S1AA_20150702T172305_20150726T172306_VVP024_INT40_G_ueF_A8ED_dem.tif
Writing:  S1AA_201

In [4]:
crop_dirs = os.listdir('/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_crop')

for idir in crop_dirs:
    cwd = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_crop/{idir}'
    os.chdir(cwd)
    allfiles = os.listdir(cwd)
    unw_fn = ''
    for fn in allfiles:
        if fn[-9:] == 'phase.tif':
            unw_fn = fn
    unw_path = f'/Users/qbren/Desktop/taco/projects/atmospheric_correction/data_processing/test_data/asc_crop/{idir}/{unw_fn}'
    src = rio.open(unw_path)
    print(src.shape)
    src.close()   